In [1]:
import pandas as pd
import pandahouse as ph
import numpy as np
import seaborn as sns
import hashlib
from scipy import stats

In [1]:
connection = {    }

### Linearisation method

In [2]:
q3 = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2022-12-02' and '2022-12-08'
    and exp_group in (1,2)
GROUP BY exp_group, user_id
"""
df_ll= ph.read_clickhouse(q3, connection=connection)
df_ll.head()

NameError: name 'ph' is not defined

In [25]:
df_llc=df_ll[df_ll.exp_group==1]
CTR_c=sum(df_llc.likes)/sum(df_llc.views)
CTR_c

0.20802680490126244

In [26]:
df_ll['lin_likes']=df_ll.likes-df_ll.views*CTR_c
df_ll.head()

,exp_group,user_id,likes,views,ctr,lin_likes
0,1,109963,3,15,0.200000,-0.120402
1,1,26117,32,141,0.226950,2.668221
2,1,138232,18,73,0.246575,2.814043
3,1,26295,33,122,0.270492,7.620730
4,1,18392,7,32,0.218750,0.343142


In [27]:
stats.ttest_ind(df_ll[df_ll.exp_group == 1].ctr,
                df_ll[df_ll.exp_group == 2].ctr,
                equal_var=False)

Ttest_indResult(statistic=0.7094392041270485, pvalue=0.4780623130874935)

In [28]:
stats.ttest_ind(df_ll[df_ll.exp_group == 1].lin_likes,
                df_ll[df_ll.exp_group == 2].lin_likes,
                equal_var=False)

Ttest_indResult(statistic=6.122579994775972, pvalue=9.439432187037712e-10)

In [29]:
q4 = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2022-12-02' and '2022-12-08'
    and exp_group in (0,3)
GROUP BY exp_group, user_id
"""
df_ll1= ph.read_clickhouse(q4, connection=connection)
df_llc1=df_ll1[df_ll1.exp_group==0]
CTR_c1=sum(df_llc1.likes)/sum(df_llc1.views)
df_ll1['lin_likes']=df_ll1.likes-df_ll1.views*CTR_c1

In [30]:
stats.ttest_ind(df_ll1[df_ll1.exp_group == 0].ctr,
                df_ll1[df_ll1.exp_group == 3].ctr,
                equal_var=False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [31]:
stats.ttest_ind(df_ll1[df_ll1.exp_group == 0].lin_likes,
                df_ll1[df_ll1.exp_group == 3].lin_likes,
                equal_var=False)

Ttest_indResult(statistic=-15.21499546090383, pvalue=5.491424947968532e-52)